## Loading Images

In [2]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Image dimensions
img_width, img_height = 64, 64

# Directory paths
train_dir = 'dataset/Gambo/Train'
test_dir = 'dataset/Gambo/Test'

# Create an instance of ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(rescale=1.0/255.0)  # Normalize pixel values to [0, 1]

# Load 1000 images from each class (Normal and Reversal) for training
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=32,  # You can choose a batch size; this will load 32 images at a time
    class_mode='binary',  # Binary classification: Normal (0), Reversal (1)
    classes=['Normal', 'Reversal'],  # Specify class subdirectories
    shuffle=True,
    subset=None,
)

# Load 1000 images from each class (Normal and Reversal) for testing
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='binary',
    classes=['Normal', 'Reversal'],
    shuffle=False,  # No need to shuffle during testing
    subset=None,
)


Found 86115 images belonging to 2 classes.
Found 37439 images belonging to 2 classes.


## Training the ANN Model

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

# Build the ANN model
model = Sequential()

# Flatten the image from 64x64x3 (RGB) to a single vector
model.add(Flatten(input_shape=(img_width, img_height, 3)))

# Add dense layers
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))

# Output layer (for binary classification: Normal/Reversal)
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()


c:\Users\baaqa\miniconda3\envs\tf-cpu\lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 12288)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,572,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,581,313 (6.03 MB)

 Trainable params: 1,581,313 (6.03 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=2000 // train_generator.batch_size,  # Total 2000 images (1000 Normal, 1000 Reversal)
    epochs=10,  # You can adjust the number of epochs based on performance
    validation_data=test_generator,
    validation_steps=2000 // test_generator.batch_size  # Total 2000 images (1000 Normal, 1000 Reversal)
)


c:\Users\baaqa\miniconda3\envs\tf-cpu\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 59s 926ms/step - accuracy: 0.6869 - loss: 0.6394 - val_accuracy: 0.8831 - val_loss: 0.3241
Epoch 2/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 54s 883ms/step - accuracy: 0.8220 - loss: 0.3871 - val_accuracy: 0.7303 - val_loss: 0.5408
Epoch 3/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 53s 865ms/step - accuracy: 0.8824 - loss: 0.2754 - val_accuracy: 0.9849 - val_loss: 0.0572
Epoch 4/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 51s 839ms/step - accuracy: 0.8979 - loss: 0.2655 - val_accuracy: 0.8332 - val_loss: 0.4481
Epoch 5/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 44s 715ms/step - accuracy: 0.9142 - loss: 0.2228 - val_accuracy: 0.8977 - val_loss: 0.3549
Epoch 6/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 30s 494ms/step - accuracy: 0.9302 - loss: 0.1843 - val_accuracy: 0.8427 - val_loss: 0.4516
Epoch 7/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 24s 397ms/step - accuracy: 0.9368 - loss: 0.1644 - val_accuracy: 0.9178 - val_loss: 0.2282
Epoch 8/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 28s 459ms/step - accuracy: 0.9321 - loss: 0.1747 - val_accu

## Evaluation

In [5]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {accuracy * 100:.2f}%')


1170/1170 ━━━━━━━━━━━━━━━━━━━━ 297s 254ms/step - accuracy: 0.8284 - loss: 0.4664
Test Accuracy: 83.96%


## Making Predictions

In [10]:
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_image(img_path, model):
    img = image.load_img(img_path, target_size=(img_width, img_height))
    img_array = image.img_to_array(img) / 255.0  # Normalize the image
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    
    # Predict the class
    prediction = model.predict(img_array)
    if prediction < 0.5:
        print("Normal")
    else:
        print("Reversal")

# Test on a new image
for i in range(900,921):
    predict_image(f"1_{i}.png", model)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Reversal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Reversal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Reversal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
Reversal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Reversal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Reversal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Reversal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Normal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Reversal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Reversal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Normal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Reversal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Reversal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Reversal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Reversal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Reversal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Reversal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Reversal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Normal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Reversal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
Reversal


## Save the model

In [12]:
# Save the trained model
model.save('pattern_classifier.h5')
